In [51]:
# import koco
# dataset = koco.load_dataset('korean-hate-speech', mode='train_dev')
# testset = koco.load_dataset('korean-hate-speech', mode='test')

In [52]:
# len(dataset['train']), len(dataset['dev'])

In [53]:
# dataset['train'][0].keys()

In [54]:
# dataset['train'][0]['comments']

In [55]:
# train_dataset = dataset['train']
# eval_dataset = dataset['dev']

### comment preprocessing
- 숫자는 욕설로 만들 수 있음
- 초성만으로 이루어진 단어또한 비웃음이나 욕설로 만들 수 있음
- 이모티콘은 실제 사용에서는 어떨지 모르나 데이터로 들어올 때는 학습에 방해될 것으로 예상됨(UNK 토큰으로 판단될 수 있다고 생각함)

In [56]:
""" 
    Reference : https://gist.github.com/kse0202/9d3d8d519170064cefdd12fcb718afa0
"""
import re
def preprocessing(dataset):
    for i in range(len(dataset)):
        comment = dataset['text'][i]
        comment = re.sub(r'([♡❤✌❣♥ᆢ✊❤️✨⤵️☺️;”“☝●✔️ㆍ„♨⁉° ͜ʖ͡°╭∩╮ɑ]+)', ' ', comment)
        comment = re.sub(r'[...]+','',comment)
        emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                                    "]+", flags=re.UNICODE)
        comment=  emoji_pattern.sub(r'', comment) # 유니코드로 이모티콘 지우기
        dataset['text'][i] = comment
    return dataset

In [57]:
import pandas as pd
train_dataset = pd.read_csv('tmp_curse.csv')

In [58]:
train_dataset = preprocessing(train_dataset)
# eval_dataset = preprocessing(eval_dataset)

/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
# print(train_dataset[0]['comments'])
print(train_dataset['text'][0])
print(train_dataset['curse'][0])

좌배 까는건 ㅇㅂ
0


In [60]:
# label_to_num = dict(
#     none = 0, hate = 1
# )
# label_to_num


In [61]:
label_to_num = dict(
    none = 0, curse = 1
)
label_to_num

{'none': 0, 'curse': 1}

In [62]:
num_to_label = {}
for k in label_to_num.keys():
    num_to_label[label_to_num[k]]=k
num_to_label

{0: 'none', 1: 'curse'}

In [63]:
# import pickle
# with open('./label_to_num', 'rb') as f:
#     label_to_num = pickle.load(f)

# with open('./num_to_label', 'rb') as f:
#     num_to_label = pickle.load(f)

In [64]:
import pandas as pd
from tqdm import trange

def tocsv(dataset):
    df = pd.DataFrame(columns=['text','label'])
    for i in trange(len(dataset)):
        df=df.append(
            dict(
                text=dataset['text'][i],
                label=(0 if dataset['curse'][i]==0 else 1)
            )
            ,ignore_index=True)
    return df

In [65]:
train_df = tocsv(train_dataset)
# eval_df = tocsv(eval_dataset)

100%|██████████| 1107/1107 [00:02<00:00, 455.50it/s]


In [66]:
print('train dataset none 개수 : {}, curse 개수 : {}'.format(list(train_df['label']).count(0), list(train_df['label']).count(1)))
# print('eval dataset none 개수 : {}, hate 개수 : {}'.format(list(eval_df['label']).count(0), list(eval_df['label']).count(1)))

train dataset none 개수 : 853, curse 개수 : 254


In [67]:
train_df.to_csv('./curse.csv')
# eval_df.to_csv('./eval_data.csv')

토크나이저를 학습하지 말고 사전학습된 토크나이저를 사용하자.  
  
왜냐하면 문장내 어떤 단어가 올지 모르겠음 -> 단어가 더 많은 사전학습된 토크나이저가 유리하다고 판단  
  
또한, 전처리 부분에 있어서도 사전학습된 토크나이저가 성능이 좋은듯  
  
하지만 욕설에 대해서 토크나이징이 어떨지는 테스트해야할듯

In [68]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("emeraldgoose/bad-korean-tokenizer")
tokenizer

PreTrainedTokenizerFast(name_or_path='emeraldgoose/bad-korean-tokenizer', vocab_size=50135, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [69]:
tokenizer.get_added_vocab()

{'꽆고': 50139,
 '뮫으면': 50138,
 '띙구렇게': 50137,
 '이라는둥': 50136,
 '껓뱀': 50135,
 '참지않긬': 50141,
 '재믺나요': 50140}

In [70]:
unk_ = []
for text in list(train_df['text']):
    tokens = tokenizer(text)['input_ids']
    if 1 in tokens:
        unk_.append([text, tokenizer.decode(tokens)])

In [71]:
unk_

[['싸캐스틱이다 / ː /', '[CLS] 싸캐스틱이다 / [UNK] / [SEP]'],
 ['집 그리 좋아보이진않는데 장인이 일도 못하게 한다고? 얼마정도ᄇᆞ는데 장인이',
  '[CLS] 집 그리 좋아보이진않는데 장인이 일도 못하게 한다고? [UNK] 장인이 [SEP]'],
 ['여친먹는거 한 입 먹어밨는데 좀,,,,국물은 갠탆앗음',
  '[CLS] 여친먹는거 한 입 먹어밨는데 좀,,,, 국물은 [UNK] [SEP]']]

In [73]:
adding = ['갠탆앗음', '얼마정도ᄇᆞ는데']
tokenizer.add_tokens(adding)

2

In [76]:
tokenizer.get_added_vocab()

{'꽆고': 50139,
 '뮫으면': 50138,
 '갠탆앗음': 50142,
 '띙구렇게': 50137,
 '이라는둥': 50136,
 '얼마정도ᄇᆞ는데': 50143,
 '껓뱀': 50135,
 '참지않긬': 50141,
 '재믺나요': 50140}

## 추가해야 할 단어들
- 갠탆앗음
- 얼마정도ᄇᆞ는데
